In [1]:
# library imports 

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, log_loss
import numpy as np

In [2]:
# regression data
import random
from sklearn.datasets import make_classification

# rng
random.seed(0)

regression_params = {
     'n_samples':1000
    ,'n_features':5
    ,'n_informative':3
    ,'n_redundant':0
    ,'n_repeated':0
    ,'n_classes' : 4
    ,'n_clusters_per_class':1
    ,'random_state':0
    ,'class_sep' : .2
}

X,y = make_classification(**regression_params)

# add constant
X = np.concatenate([X,np.ones(shape=(len(X),1))],axis=1)

# train / test splits
X_train,X_test,y_train,y_test = train_test_split(X,y)

# statsmodels

In [3]:
import statsmodels.api as sm

In [4]:
sm_model = sm.MNLogit(endog=y_train,exog=X_train)

In [5]:
result = sm_model.fit_regularized(maxiter=1000)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2984772059099112
            Iterations: 41
            Function evaluations: 41
            Gradient evaluations: 41


In [6]:
print(result.summary())

                          MNLogit Regression Results                          
Dep. Variable:                      y   No. Observations:                  750
Model:                        MNLogit   Df Residuals:                      732
Method:                           MLE   Df Model:                           15
Date:                Tue, 06 Jul 2021   Pseudo R-squ.:                 0.06269
Time:                        21:47:01   Log-Likelihood:                -973.86
converged:                       True   LL-Null:                       -1039.0
Covariance Type:            nonrobust   LLR p-value:                 1.877e-20
       y=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0546      0.113     -0.481      0.630      -0.277       0.168
x2            -0.5928      0.114     -5.214      0.000      -0.816      -0.370
x3             0.0138      0.159      0.087      0.9

In [7]:
# accuracy
(np.argmax(result.predict(X_test),axis=1) == y_test).mean()

0.36

In [8]:
# roc auc
roc_auc_score(y_test,result.predict(X_test),multi_class='ovr')

0.6653827462996299

# sklearn

In [9]:
from sklearn.linear_model import LogisticRegression

In [10]:
sk_model = LogisticRegression(multi_class='ovr')

In [11]:
sk_model = sk_model.fit(X_train[:,0:-1],y_train)

In [12]:
sk_model.coef_

array([[-2.23386574e-01,  4.61901402e-01,  1.62301064e-01,
        -1.70926615e-01, -8.73495811e-03],
       [-3.37800917e-01, -2.93954707e-01,  2.28866446e-01,
         2.85046178e-02, -1.81787090e-02],
       [ 3.91953369e-01, -3.92626338e-01, -8.88712087e-01,
        -2.73145394e-04, -4.63002598e-03],
       [ 1.72442010e-01,  2.53986542e-01,  4.75584792e-01,
         1.33588486e-01,  2.18090386e-02]])

In [13]:
sk_model.intercept_

array([-1.27373737, -1.05333191, -1.15251165, -1.21814538])

In [14]:
# accuracy
(sk_model.predict(X_test[:,0:-1]) == y_test).mean()

0.36

In [15]:
# roc auc
roc_auc_score(y_test,sk_model.predict_proba(X_test[:,0:-1]),multi_class='ovr')

0.6658679362052116

# pure python

In [16]:
# turn back into numpy arrays

X_train = X_train
y_train = y_train.reshape(-1,1)

In [17]:
# functions

def sigmoid(x):
    return 1/(1+np.exp(-x))

def predict(x,beta):
    return sigmoid(np.dot(x,beta))

def gradient_step(beta,grad,step_size=0.01):
    return beta - step_size*grad

def gradient(x,beta,y_true):
    y_pred = predict(x,beta)
    
    return np.dot(np.transpose(x),(y_pred-y_true))/len(x)

In [18]:
betas = np.zeros(shape=(regression_params['n_features']+1,regression_params['n_classes']))
for _class in np.unique(y):
    betas[:,_class] = [random.random() for _ in range(len(betas))]
betas

array([[0.84442185, 0.78379859, 0.28183784, 0.81021724],
       [0.7579544 , 0.30331273, 0.7558042 , 0.90216595],
       [0.42057158, 0.47659695, 0.618369  , 0.31014757],
       [0.25891675, 0.58338204, 0.25050634, 0.72983175],
       [0.51127472, 0.90811289, 0.90974626, 0.89883829],
       [0.40493414, 0.50468686, 0.98278548, 0.68398393]])

In [19]:
epochs = 100000
for _class in np.unique(y):
    beta = betas[:,_class].reshape(-1,1)
    for epoch in range(epochs):
        grad = gradient(x=X_train,beta=beta,y_true=(y_train==_class).astype('int'))
        beta = gradient_step(beta=beta,grad=grad,step_size=0.1)
    betas[:,_class] = beta.flat[:]

In [20]:
betas

array([[-2.26282818e-01, -3.40409617e-01,  3.96983992e-01,
         1.73491619e-01],
       [ 4.66551597e-01, -2.95727059e-01, -3.98229965e-01,
         2.56867153e-01],
       [ 1.66542977e-01,  2.32308357e-01, -9.07153792e-01,
         4.83763461e-01],
       [-1.72496044e-01,  2.88923931e-02, -2.58895962e-04,
         1.34485278e-01],
       [-8.55878637e-03, -1.83972324e-02, -5.49724574e-03,
         2.23325148e-02],
       [-1.27558545e+00, -1.05421466e+00, -1.15479799e+00,
        -1.22037716e+00]])

In [21]:
# accuracy
(np.argmax(predict(X_test,betas),axis=1) == y_test).mean()

0.36

In [22]:
# roc auc
roc_auc_score(y_test,(predict(X_test,betas) /predict(X_test,betas).sum(axis=1).reshape(-1,1)),multi_class='ovr')

0.6658436502697178